# What about the Attrition?

*An analysis based on IBM HR Dataset - Provided by kaggle.com*

<img src='https://www.pointillist.com/wp-content/uploads/2017/11/Running-4-the-Exit-2-844x422-800x400.png'>

Organizations are maded of people. Inside the company, we can find individuals with different backgrounds, experiences and  with different objetives. 
Sometimes, what the individual wants for him or her career, is aligned with the organization objetives. In that case, the individual is able to pursuit his personal objetive, while following the organization one.
In other times, the organizational objetive is not aligned with the individual one, so he or she may enter in an internal conflict.

Since organizations are **people** working following a main objetive, is not crazy to affirm that they are the **most valuable** asset. People working in different possitions, add value while working, and sometimes they are the visible face of the bussiness.

When someone enter in a company, is needed to spend time in a good onboarding, trainings, and development of him or her skills. It's a big investment, and sometimes is not seen as that. Because of that, when an employee decide to **quit**, it's a **double loss**: social and economical... so it is really important to understand the main causes, and think in solutions to avoid lose our best employees.

The **new paradigm** in organizations, include people from different ages and way of thinking working side by side, new knowledge and technology, new ways of work (presential, remote, etc), and a fast enviroment never seen before.
If you want, I invite you to follow me in this trip, triying to get the first insights about **"Why people left their jobs?"**, and think in ways to don't lose our main value.

## Content Table
* [1.0 Preliminar Work](#Preliminar)
    * [1.1 Importing Libraries](#Libraries)
    * [1.2 Loading Dataset](#load)
* [2.0 First Look at the Data](#first_look)
    * [2.1 Attrition, Job Position and Education](#education)
    * [2.2 Attrition and Temporal Features](#time)
    * [2.3 What about generations?](#generations)
    * [2.4 Gender Analysis](#gender)
    * [2.5 Attrition and Job Features](#job)
* [3.0 Cluster Analysis](#cluster)
* [4.0 Predictive Model](#model)
* [5.0 Oversampling](#oversampling)

## 1.0 Preliminar Work <a class="anchor" id="Preliminar"></a>
------------------------------------------------


### 1.1 Importing Libraries <a class="anchor" id="Libraries"></a>


In [ ]:
!pip install seaborn==0.11.0

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import plotly.express as px
import plotly.graph_objs as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### 1.2 Loading the Dataset <a class="anchor" id="load"></a>


In [ ]:
df = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [ ]:
print("A first aproach to our dataset \n")
display(df.head(2), df.describe().transpose(), df.columns)


As we could see, our dataset has <mark style="background-color: lightblue"> 1470 </mark>rows , <mark style="background-color: lightblue">35 </mark>columns, and <mark style="background-color: lightblue">0</mark> null values

## 2.0 First look at the Data <a class="anchor" id="first_look"></a>
-------------------------------------------

Before to develop our model, we a
re going to make a **descriptive analysis** trying to get insights about the main causes of attrition in our personal

### 2.1 Attrition, Job Position and Education <a class="anchor" id="education"></a>

In [ ]:
fig, axis=plt.subplots(3,2,figsize=(17,15))
plt.suptitle('Attrition and Job position - Education',size=15)

sns.histplot(x=df.Department, hue=df.Attrition, ax=axis[0,0],palette="Pastel1")
axis[0,0].set_title('Employees and Atrittion per Department', size=10)
sns.histplot(df, x=df.Department, hue=df.Attrition, ax=axis[0,1],palette="Pastel1", stat="probability", multiple="fill", shrink=.8)

sns.histplot(x=df.Education, hue=df.Attrition, ax=axis[1,0],bins=5,palette="Pastel1")
axis[1,0].set_title('Employees and Attrition per Education Level', size=10)
sns.histplot(df, x=df.Education, hue=df.Attrition, ax=axis[1,1],bins=5,palette="Pastel1", stat="probability", multiple="fill", shrink=.8)

sns.histplot(x=df.EducationField, hue=df.Attrition, ax=axis[2,0],palette="Pastel1")
axis[2,0].set_title('Employees and Attrition per Education Field', size=10)
axis[2,0].set_xticklabels(labels=list(df.EducationField.unique()),rotation=30)
sns.histplot(df, x=df.EducationField, hue=df.Attrition, ax=axis[2,1],palette="Pastel1", stat="probability", multiple="fill", shrink=.8)
axis[2,1].set_xticklabels(labels=list(df.EducationField.unique()),rotation=30)
;


As we can see in the first subplots, in our dataset we have more people related with the **Research and Development Area**, with mid-high levels of educations, and they are from fields related with **Life Sciences and Medicine**.
But when we analyze the proportion of attrition in each category, we discover that the education level is not so important as the Education field or the Area. More people working in Sales or Human Resouces showed attrition, for example.
But as we know that we have less observations in the categories with high proportion of attrition, <mark style="background-color: lightblue">we are not going to take fast conclussions</mark>.

### 2.2 Attrition and temporal features <a class="anchor" id="time"></a>

In [ ]:
fig, axis=plt.subplots(5,2,figsize=(17,15))
plt.suptitle('Attrition and Temporal Features',fontsize=20)
sns.histplot(x=df.TotalWorkingYears, hue=df.Attrition, ax=axis[0,0],palette='Pastel1',multiple="stack")
axis[0,0].set_title('Total Working Years', size=10)
sns.histplot(df, x=df.TotalWorkingYears, hue=df.Attrition, ax=axis[0,1],palette="Pastel1", stat="probability", multiple="fill", shrink=.8)

sns.histplot(x=df.YearsAtCompany, hue=df.Attrition, ax=axis[1,0],palette='Pastel1',multiple="stack")
axis[1,0].set_title('Years at Company', size=10)
sns.histplot(df, x=df.YearsAtCompany, hue=df.Attrition, ax=axis[1,1],palette="Pastel1", stat="probability", multiple="fill", shrink=.8)

sns.histplot(x=df.YearsInCurrentRole, hue=df.Attrition, ax=axis[2,0],palette='Pastel1',multiple="stack")
axis[2,0].set_title('Years in Current Role', size=10)
sns.histplot(df, x=df.YearsInCurrentRole, hue=df.Attrition, ax=axis[2,1],palette="Pastel1", stat="probability", multiple="fill", shrink=.8, bins=19)

sns.histplot(x=df.YearsSinceLastPromotion, hue=df.Attrition, ax=axis[3,0],palette='Pastel1',multiple="stack")
axis[3,0].set_title('Years since last promotion', size=10)
sns.histplot(df, x=df.YearsSinceLastPromotion, hue=df.Attrition, ax=axis[3,1],palette="Pastel1", stat="probability", multiple="fill", shrink=.8,bins=16)

sns.histplot(x=df.YearsWithCurrManager, hue=df.Attrition, ax=axis[4,0],palette='Pastel1',multiple="stack")
axis[4,0].set_title('Years with current manager', size=10)
sns.histplot(df, x=df.YearsWithCurrManager, hue=df.Attrition, ax=axis[4,1],palette="Pastel1", stat="probability", multiple="fill", shrink=.8,bins=18)

plt.subplots_adjust(hspace=1)

Comparing the attrition with temporal features, we could notice:

**Total Working Years**: as we can imagine, we have 100% of attrition at the level of 40 total working years, maybe beacuse at that time, people start to retire.
But, we can also notice an intersting number of people who quit between the 0 and 5 total working years. Maybe we are talking about young proffesionals, who wants to adquire experience in different companies.

### 2.3 What about generations? <a class="anchor" id="generations"></a>

Which Generation are You?


|Generation Name | Births Start | Births End	| Youngest Age | Oldest Age |
| :-: | :-: | :-: | :-: | :-: |
|The Silent Generation | 1928 | 1945 | 71 | 88 |
|Baby Boomer | 1946 | 1964 | 52 | 70 |
|Generation X | 1965 | 1980 | 36 | 51 |
|Millenials | 1981 | 1999 | 17 | 35 |
|Gen Z | 2000 | 2020 | 0 | 16 |

(*age if still alive on 2016, year when the data was taken)

In [ ]:
# We will build a new feature, which classifies the age in the main "Generations" accepted by theory.
df['Generation'] = df.Age.apply(lambda x: 'Millenials' if (x>=17) and (x<36) else('Generation X' if (x>=36) and (x<52) else ('Baby Boomers' if (x>=52) and (x<71) else ('Silent Generation' if (x>=71) and (x<89) else x))))

In [ ]:
fig = plt.figure(1)
plt.suptitle('What About Generations?',fontsize=20)
# set up subplot grid
gridspec.GridSpec(3,3)

# large subplot
plt.subplot2grid((3,3), (0,0), colspan=1, rowspan=3)
plt.locator_params(axis='x', nbins=5)
plt.locator_params(axis='y', nbins=5)
plt.title('Generations by Job Position')
plt.xlabel('Number of People')
plt.ylabel('Generation')
sns.histplot(y=df.JobRole, hue=df.Generation, palette='Pastel1',multiple="stack")

# small subplot 1
plt.subplot2grid((3,3), (0,1),colspan=2)
plt.locator_params(axis='x', nbins=5)
plt.locator_params(axis='y', nbins=5)
plt.title('Generations Inside the Company')
plt.xlabel('Age')
plt.ylabel('Employees')
sns.histplot(x=df.Age, hue=df.Generation,palette='Pastel1')

# small subplot 2
plt.subplot2grid((3,3), (1,1),colspan=2)
plt.locator_params(axis='x', nbins=5)
plt.locator_params(axis='y', nbins=5)
plt.title('Years in company per Generation')
plt.xlabel('Years in Company')
plt.ylabel('Employees')
sns.histplot(x=df.YearsAtCompany, hue=df.Generation, palette='Pastel1',multiple="stack")

# small subplot 3
plt.subplot2grid((3,3), (2,1),colspan=2)
plt.locator_params(axis='x', nbins=5)
plt.locator_params(axis='y', nbins=5)
plt.title('Generations and Monthly income')
plt.xlabel('Generations')
plt.ylabel('Income')
sns.boxplot(x=df.Generation, y=df.MonthlyIncome, hue=df.Attrition,palette='Pastel1')

# fit subplots and save fig

fig.set_size_inches(w=20,h=10)
plt.subplots_adjust(hspace=.5)
plt.show()

### 2.4 Gender Analysis <a class="anchor" id="gender"></a>

I will like to think that genre is not an important feature while we are studying the labor force (since all of us are equal). But the reality is we still having a long road to walk in the route of equality, so is important to compare an analyse some features between genres, to find inequities and develop plans to solve it.
Some questions that I thought:

¿How is the labor force distributed?

¿How is the distribution of income and job positions?

¿Are there differences between men and women related with attrition?

In [ ]:
fig = plt.figure(1)
plt.suptitle('Gender Analysis',fontsize=30,x = .5)
# set up subplot grid
gridspec.GridSpec(3,3)

# large subplot
plt.subplot2grid((3,3), (0,0), colspan=1, rowspan=3)
plt.title('Gender by Job Position')
plt.xlabel('Number of People')
plt.ylabel('Gender')
sns.histplot(y=df.JobRole, hue=df.Gender, palette='Pastel1',multiple="stack")

plt.subplot2grid((3,3), (0,1),colspan=1)
plt.title('Gender Distribution')
labels = list(df.groupby("Gender")["EmployeeCount"].sum().keys())
plt.pie(x=df.groupby("Gender")["EmployeeCount"].sum(), 
        autopct="%.1f%%",  labels=labels,explode=[0.05]*2, shadow=True)

plt.subplot2grid((3,3), (0,2),colspan=1)
plt.title('Attrition By Gender')
labels = list(df.loc[df['Attrition']=='Yes'].groupby("Gender")["EmployeeCount"].sum().keys())
plt.pie(x=df.loc[df['Attrition']=='Yes'].groupby("Gender")["EmployeeCount"].sum(), 
        autopct="%.1f%%",  labels=labels, radius=1,pctdistance=0.5, explode=[0.05]*2, shadow=True)

plt.subplot2grid((3,3), (2,1),colspan=1)
plt.title('Monthly Income By Gender')
sns.boxplot(data=df,x='Gender', y='MonthlyIncome', palette='Pastel1')

plt.subplot2grid((3,3),(1,1),colspan=1)
plt.title('Years since last promotion')
sns.histplot(x=df.YearsSinceLastPromotion,hue=df.Gender, palette='Pastel1', multiple='stack',bins=16)

plt.subplot2grid((3,3), (1,2), colspan=1)
plt.title('WorkLife Balance')
sns.histplot(x=df.WorkLifeBalance,hue=df.Gender, palette='Pastel1', bins=4,multiple='stack')

plt.subplot2grid((3,3),(2,2),colspan=1)
plt.title('Marital Status')
sns.histplot(x=df.MaritalStatus, hue=df.Gender, palette='Pastel1',multiple='stack')

fig.set_size_inches(w=20,h=15)
plt.subplots_adjust(hspace=.2)
plt.show()

After the analysis, we didn't see diferences between gender if we talk about income, job posittions or attrition. Instead of that, we see more men in positions like Laboratory, Sales, Human Resources and Research.

Something to take note, is that the proportion of women is less than men, in the total of employees. We don't have more information to make conclussions about that.

### 2.5 Attrition and Job features <a class="anchor" id="job"></a>

In [ ]:
fig = plt.figure(1)
plt.suptitle('Job Analysis',fontsize=30,x = .5)
# set up subplot grid
gridspec.GridSpec(3,3)

# large subplot
plt.subplot2grid((3,3), (0,0), colspan=1)
plt.title('Attrition and Salary')
plt.xlabel('Attrition')
plt.ylabel('MonthlyIncome')
sns.boxenplot(y=df.MonthlyIncome,x=df.Attrition, palette='Pastel1')

plt.subplot2grid((3,3), (0,1), colspan=2)
plt.title('Attrition and Job Level')
plt.xlabel('Job Level')
plt.ylabel('People')
sns.countplot(data=df, x='JobLevel', hue='Attrition',palette='Pastel1')

plt.subplot2grid((3,3), (1,0), colspan=1)
plt.title('Attrition , Overtime and Monthly Income')
plt.xlabel('Overtime')
plt.ylabel('People')
sns.boxenplot(data=df, x='OverTime',y='MonthlyIncome', hue='Attrition',palette='Pastel1')

plt.subplot2grid((3,3), (1,1),colspan=1)
plt.title('Attrition in OverTime Yes')
labels = list(df.loc[df['OverTime']=='Yes'].groupby("Attrition")["EmployeeCount"].sum().keys())
plt.pie(x=df.loc[df['OverTime']=='Yes'].groupby("Attrition")["EmployeeCount"].sum(), 
        autopct="%.1f%%",  labels=labels, radius=1,pctdistance=0.5, explode=[0.05]*2, shadow=True)

plt.subplot2grid((3,3), (1,2),colspan=1)
plt.title('Attrition in OverTime No')
labels = list(df.loc[df['OverTime']=='No'].groupby("Attrition")["EmployeeCount"].sum().keys())
plt.pie(x=df.loc[df['OverTime']=='No'].groupby("Attrition")["EmployeeCount"].sum(), 
        autopct="%.1f%%",  labels=labels, radius=1,pctdistance=0.5, explode=[0.05]*2, shadow=True)

plt.subplot2grid((3,3), (2,0), colspan=1)
plt.title('Attrition and Job Satisfaction')
plt.xlabel('Job Satisfaction')
plt.ylabel('People')
sns.countplot(data=df, x='JobSatisfaction', hue='Attrition',palette='Pastel1')

plt.subplot2grid((3,3), (2,1), colspan=2)
plt.title('Attrition and EnvironmentSatisfaction')
plt.xlabel('EnvironmentSatisfaction')
plt.ylabel('People')
sns.countplot(data=df, x='EnvironmentSatisfaction', hue='Attrition',palette='Pastel1')



fig.set_size_inches(w=20,h=15)
plt.subplots_adjust(hspace=.2)
plt.show()

## 3.0 Cluster Analysis<a class="anchor" id="cluster"></a>
--------

With the objetive of getting more insights about our staff, we are going to develop a no supervized ML Model, with Kmeans.
We want to analyze if there are common patters between employees and Age, Income, and Career.
Before to train the model, we created a new feature related with the time since last promotion, as a ratio of total years at company.

In [ ]:
df['LastPromotion_ratio']=  (df['YearsSinceLastPromotion'] / df['YearsAtCompany']).round(2)
       

In [ ]:
df=df.fillna(value=0)

In [ ]:
X_cluster=df[['MonthlyIncome','Age','LastPromotion_ratio']]
#Because we work with distances, at the very beginning we have to standarize the features.
scaler=StandardScaler()
X_cluster_scaled=scaler.fit_transform(X_cluster);


In [ ]:
sq_distances=[]
k_values=range(2,10);

for k in k_values:
    kmeans=KMeans(n_clusters=k,n_init=10,random_state=0) # Numero de repeticiones=10
    kmeans.fit(X_cluster_scaled)
    sq_distances.append(kmeans.inertia_)

sns.lineplot(x=k_values,y=sq_distances,marker='o',size=30,legend=False);
plt.ylabel('Sum quadratic distance');plt.xlabel('Number of clusters');    

In [ ]:
from sklearn.metrics import silhouette_score,silhouette_samples

sil=[]
k_values=range(2,10);

for k in k_values:
    kmeans=KMeans(n_clusters=k,n_init=10,random_state=0) # Numero de repeticiones=10
    kmeans.fit(X_cluster_scaled)
    score=silhouette_score(X_cluster_scaled,kmeans.labels_)
    sil.append(score)

sns.lineplot(x=k_values,y=sil,marker='o',size=30,legend=False);
plt.ylabel('Silhouette score',fontsize=15);plt.xlabel('Number of clusters',fontsize=15);

After the analysis with Silhouette Score and Square Distances, we decided to choose 3 clusters as the optimum to train our KMeans Model.

In [ ]:
kmeans=KMeans(n_clusters=3,n_init=10,random_state=0) # Number of rep=10
kmeans.fit(X_cluster_scaled)

In [ ]:
labels=kmeans.labels_

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=X_cluster_scaled[:,0],y=X_cluster_scaled[:,2],z=X_cluster_scaled[:,1],mode='markers',
                                   marker = dict( size = 7, 
                                                 color = labels, 
                                               colorscale ='Viridis', 
                                               opacity = 0.6
                                                ))])
fig.update_layout(title=dict(text="Cluster Analysis", x=0.5,y=.98),
                   scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",
                         title='MonthlyIncome',),
                    yaxis = dict(
                        backgroundcolor="rgb(230, 200,230)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white",
                        title='Last Promotion Ratio',),
                    zaxis = dict(
                        backgroundcolor="rgb(230, 230,200)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white",
                        title='Age',),),
                    width=600,
                    height=600,
                    margin=dict(
                    r=0, l=0,
                    b=10, t=30)
                  )
fig.show()

As we can see in the graph, we have three different clusters, represented by:


|Cluster|Color|Meaning|
| :-: | :-: | :-: |
|Cluster 1| Purple| People from different ages, with low monthly income and a low promotion ratio*.|
|Cluster 2| Green | People from different ages, with low montlhy income and high promotion ratio.|
|Cluster 3| Yellow| People with higher age, good income and different promotion ratios.|

<mark style="background-color: lightblue">Cluster 1</mark>, could be related with people who is building their careers. So, they have benn promoted recently, but they still having low income. With that cluster, we may focus in the satisfaction with the new rol, and with the new knowledge.

<mark style="background-color: lightblue">Cluster 2</mark>, keeps people with low income and with a high promotion ratio, what it means that they hadn't been promoted since a long time. We must pay attention here, and take notes about their feelings related with the enviroment, company, etc. because they may be unmotivated or not happy with their situation.

<mark style="background-color: lightblue">Cluster 3</mark>, with people from higher ages and good salary. They could be managers or directors, with a lot of expertice and knowledge about the bussines. The risk here, is to keep them motivated with new challanges, because if they left we may lost a lot of gained knowledge.

*A low promotion ratio implies that someone has been promoted recently.

## Predictive Model <a class="anchor" id="model"></a>
-----------

In [ ]:
### We will create a copy of our main DF, just in case
df_model = df.copy()

In [ ]:
# We drop the features less important for our analysis
df_model = df_model.drop(['EmployeeNumber','HourlyRate',
                          'MonthlyRate','DailyRate','Over18','StandardHours',
                          'Generation','EmployeeCount'], axis=1)

In [ ]:
# We convert the feature attrition to our target.
df_model['Attrition'] = df_model.Attrition.apply(lambda x: 0 if x=='No' else 1)

#### Generation of Dummies Variables

In [ ]:
df_model = pd.get_dummies(df_model, drop_first=True)

In [ ]:
# A quick look to our new variables
df_model.columns

In [ ]:
# We create the array of target, and our dataset of features
X_model= df_model.drop(['Attrition'], axis=1)
y_model= df_model.Attrition

Split the dataset between train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.30, random_state=42, stratify=y_model)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
cv = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)

In [ ]:
# we create a function to evaluate our model
def model_performance(model, name, X, y, cv):
    s = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
    print("Performance of {}:\t{:0.3} ± {:0.3}".format( \
        name, s.mean().round(3), s.std().round(3)))

In [ ]:
#Importing the base model
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=1)

model_performance(dt,"Tree base", X_train, y_train, cv)

In [ ]:
ab = AdaBoostClassifier(base_estimator=dt, n_estimators=500,random_state=1)
model_performance(ab, "AdaBoostClassifier",  X_train, y_train, cv)

In [ ]:
gb = GradientBoostingClassifier()
model_performance(gb, "GradientBoostingClassifier", X_train, y_train, cv)

In [ ]:
from sklearn.model_selection import GridSearchCV

params_ab = {"n_estimators": [500,1000],
          "learning_rate":[0.01, 0.1],
        "base_estimator__max_depth": [1, 2, 3]}

grid_ab = GridSearchCV(AdaBoostClassifier(base_estimator=dt,random_state=1), 
                       param_grid=params_ab, cv=cv, verbose=1, n_jobs=3)


In [ ]:
grid_ab.fit(X_train, y_train)

In [ ]:
model_performance(grid_ab.best_estimator_,  "AdaBoostClassifier + GS", X_train, y_train, cv)

In [ ]:
params_gb = {'n_estimators':[500, 1000] , 
             'learning_rate':[0.001, 0.001, 0.1],
            'max_depth' : [1, 2, 3, 4]}

grid_gb = GridSearchCV(gb, param_grid=params_gb, cv=cv, verbose=1, n_jobs=3)

In [ ]:
grid_gb.fit(X_train, y_train)

In [ ]:
model_performance(grid_gb.best_estimator_,  "AdaBoostClassifier + GS", X_train, y_train, cv)

## ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score


In [ ]:
from scikitplot.metrics import plot_roc
plot_roc(y_test,grid_gb.predict_proba(X_test))

In [ ]:
plot_roc(y_test,grid_ab.predict_proba(X_test))

In [ ]:
xgb_auc = roc_auc_score(y_test,grid_gb.predict(X_test))
print("AUC Value is: ", xgb_auc)

In [ ]:
xab_auc = roc_auc_score(y_test,grid_ab.predict(X_test))
print("AUC Value for Xab is: ", xab_auc)

## Feature Importance

In [ ]:
model = grid_gb.best_estimator_
model

In [ ]:
importances = model.feature_importances_
importances

In [ ]:
index = np.argsort(importances)[::-1]
names = X_model.columns[index]

plt.figure(figsize=(15,10))

plt.title("Feature Importance")


plt.bar(range(X_model.shape[1]), importances[index],color='rrrggggggggggggggggggggggggggggggggggggggg')

plt.xticks(range(X_model.shape[1]), names, rotation=90)

# Show plot
plt.show()

## LightGBM Model

In [ ]:
import lightgbm as lgb
model_lgb = lgb.LGBMClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {  
    "n_estimators": [50,100,500],      # Number of boosted trees to fit.
    "max_depth": [1,2,3,4],            # Maximum tree depth for base learners.
    "learning_rate": [0.01, 0.1],      # Boosting learning rate (xgb’s “eta”)
}

In [ ]:
grid_lgb = GridSearchCV(model_lgb, param_grid=params, cv=cv, verbose=1, n_jobs=-1)

In [ ]:
grid_lgb.fit(X_train,y_train)

In [ ]:
model_performance(grid_lgb.best_estimator_,"Light GBM", X_train, y_train, cv)


In [ ]:
lgbm_auc = roc_auc_score(y_test,grid_lgb.predict(X_test))
print("AUC value is: ", lgbm_auc)

In [ ]:
plot_roc(y_test,grid_lgb.predict_proba(X_test), plot_micro = False, plot_macro= False);

In [ ]:
lgb.plot_importance(grid_lgb.best_estimator_, max_num_features=5);

## Confussion Matrix

In [ ]:
modelgb = grid_gb.best_estimator_

In [ ]:
y_pred_gb=modelgb.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred_gb)
sns.heatmap(conf_mat, annot=True, cmap='rocket_r')
plt.title(label='Confusion Matrix GB Model')

In [ ]:
modelab = grid_ab.best_estimator_

In [ ]:
y_pred_ab=modelab.predict(X_test)

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred_ab)
sns.heatmap(conf_mat, annot=True, cmap='rocket_r')
plt.title(label='Confusion Matrix AB Model')

## 5.0 Oversampling <a class="anchor" id="oversampling"></a>
----------------------------------------------

In [ ]:
from imblearn.over_sampling import RandomOverSampler

oversampler=RandomOverSampler(sampling_strategy='minority');

X_train_os,y_train_os=oversampler.fit_resample(X_train,y_train);

print('training set:')
print(y_train_os.value_counts())

print('\ntest set:')
print(y_test.value_counts())

In [ ]:
gb_os = GradientBoostingClassifier()
model_performance(gb_os, "GradientBoostingClassifier", X_train_os, y_train_os, cv)

In [ ]:
params_gb_os = {'n_estimators':[500, 1000] , 
             'learning_rate':[0.001, 0.001, 0.1],
            'max_depth' : [1, 2, 3, 4]}

grid_gb_os = GridSearchCV(gb_os, param_grid=params_gb_os, cv=cv, verbose=1, n_jobs=3)

In [ ]:
grid_gb_os.fit(X_train, y_train)

In [ ]:
model_performance(grid_gb_os.best_estimator_, "GradientBoostingClassifier", X_train_os, y_train_os, cv)

In [ ]:
plot_roc(y_test,grid_gb_os.predict_proba(X_test))

In [ ]:
model_os = grid_gb_os.best_estimator_
model_os

In [ ]:
importances_os = model_os.feature_importances_
importances_os

In [ ]:


# index variable
indices = np.argsort(importances_os)[::-1]

# fancy indexing.
names = X_model.columns[indices]

# Plot
plt.figure(figsize=(15,10))

# Title
plt.title("Feature Importance")

# Bars
plt.bar(range(X_model.shape[1]), importances_os[indices],color='rrrggggggggggggggggggggggggggggggggggggggg')

# Feature Names
plt.xticks(range(X_model.shape[1]), names, rotation=90)

# Show plot
plt.show()

In [ ]:
y_pred_gb_os = grid_gb_os.best_estimator_.predict(X_test)

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred_gb_os)
conf_mat

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred_gb_os))

# To be continued